In [11]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [12]:
data = pd.read_csv("feature_eng_cat.csv")

X = data.drop(['LEN',"Unnamed: 0"],axis=1)
y = data['LEN']



one_hot = OneHotEncoder()
one_hot.fit(X[[ 'MON','DAY', 'HR', 'WK','TAXI_ID_CAT','ORIGIN_STAND_CAT','ORIGIN_CALL_CAT']].values)

one_hot_output = one_hot.transform(X[['MON','DAY', 'HR', 'WK','TAXI_ID_CAT','ORIGIN_STAND_CAT','ORIGIN_CALL_CAT']].values)


columns=[f"MON_{i}" for i in range(1,13)] + [f"DAT_{i}" for i in range(1,32)] + [f"HR_{i}" for i in range(24)] + [f"WK_{i}" for i in range(7)] + [f"TAXI_{i}" for i in range(442)] + [f"STAND_{i}" for i in range(64)] + [f"CALL_{i}" for i in range(254)] 
one_hot_frame = pd.DataFrame(one_hot_output.toarray(),columns=columns)
df_processed = X.join(one_hot_frame)
df_processed.drop(['MON','DAY', 'HR', 'WK','TAXI_ID_CAT','ORIGIN_STAND_CAT','ORIGIN_CALL_CAT'],axis=1,inplace=True)


x_train,x_test,y_train,y_test = train_test_split(df_processed,y,test_size=0.2)

In [16]:
# 转换为Dataset数据格式
# category_list = ['TAXI_ID_CAT', 'ORIGIN_CALL_CAT', 'ORIGIN_STAND_CAT']
# for ft in category_list:
#        x_train[ft] = x_train[ft].astype('category')
#        x_test[ft] = x_test[ft].astype('category')

lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)

# 参数
params = {
    'task': 'train',
    'boosting_type': 'gbdt',  # 设置提升类型
    'objective': 'regression',  # 目标函数
    'metric': {'l2', 'auc'},  # 评估函数
    'num_leaves': 35,  # 叶子节点数
    'learning_rate': 0.1,  # 学习速率
    'feature_fraction': 0.9,  # 建树的特征选择比例
    'bagging_fraction': 0.8,  # 建树的样本采样比例
    'bagging_freq': 5,  # k 意味着每 k 次迭代执行bagging
    'verbose': 1  # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
}

# 模型训练
gbm = lgb.train(params, lgb_train, num_boost_round=500, valid_sets=lgb_eval, early_stopping_rounds=10)

/home/aaron/anaconda3/envs/cv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 3.059784 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1796
[LightGBM] [Info] Number of data points in the train set: 1339321, number of used features: 867
[LightGBM] [Info] Start training from score 731.341202
[1]	valid_0's l2: 479782	valid_0's auc: 1
Training until validation scores don't improve for 10 rounds
[2]	valid_0's l2: 474907	valid_0's auc: 1
[3]	valid_0's l2: 470973	valid_0's auc: 1
[4]	valid_0's l2: 467615	valid_0's auc: 1
[5]	valid_0's l2: 464970	valid_0's auc: 1
[6]	valid_0's l2: 462717	valid_0's auc: 1
[7]	valid_0's l2: 460656	valid_0's auc: 1
[8]	valid_0's l2: 459079	valid_0's auc: 1
[9]	valid_0's l2: 457719	valid_0's auc: 1
[10]	valid_0's l2: 456603	valid_0's auc: 1
[11]	valid_0's l2: 455386	valid_0's auc: 1
Early stopping, best iteration is:
[1]	valid_0's l2: 479782	v

In [14]:
from sklearn.metrics import mean_squared_error

y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration)
mse_value = mean_squared_error(y_test, y_pred)
print(f"RMSE = {np.sqrt(mse_value)}")

RMSE = 689.0865688175828


In [17]:
test_data = pd.read_csv("test_public_features_cat.csv")

test_data = test_data.drop(['Unnamed: 0'],axis=1)

one_hot_output = one_hot.transform(X[['MON','DAY', 'HR', 'WK','TAXI_ID_CAT','ORIGIN_STAND_CAT','ORIGIN_CALL_CAT']].values)
one_hot_frame = pd.DataFrame(one_hot_output.toarray(),columns=columns)

test_data = test_data.join(one_hot_frame)
test_data.drop(['MON','DAY', 'HR', 'WK','TAXI_ID_CAT','ORIGIN_STAND_CAT','ORIGIN_CALL_CAT'],axis=1,inplace=True)


# predict the results
ypred_result = gbm.predict(test_data)

sample = pd.read_csv("sample_xgboost.csv",index_col="TRIP_ID")

sample['TRAVEL_TIME'] = ypred_result

sample.to_csv("sample_lightgbm.csv")
